## Setings

In [19]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

os.getcwd()
version = 'PlusMinus2_v2'
raw_data_path = 'basic_info_df_modified_PlusMinus2.csv'
basic_data_path = f'{version}_basic_data.csv'
normalized_data_path = f'{version}_normalized_data.csv'

raw_data = pd.read_csv(raw_data_path)

# 1. 필요한 함수 모음, 실행

In [20]:
# 원하는 컬럼 순서 정의
desired_columns = [
    '공고번호',
    '기초금액',
    'A값',
    '투찰률(%)',
    '참여업체수',
    '예가범위',
    '공고구분표시',
    '공고제목',
    '발주처(수요기관)',
    '지역제한',
    '정답사정률(%)'
]

# DataFrame 재구성
def reorganize_dataframe(df, desired_columns):
    """
    DataFrame의 컬럼을 원하는 순서로 재배열하고, 
    숫자 컬럼들은 그대로 유지합니다.
    """
    # 12번째 이후의 컬럼들 가져오기
    remaining_columns = [col for col in df.columns if col not in desired_columns and col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    
    # 원하는 순서의 컬럼과 나머지 컬럼들 합치기
    final_columns = desired_columns + remaining_columns
    # 새로운 순서로 DataFrame 재구성
    reorganized_df = df[final_columns]
    return reorganized_df


def normalize_and_transform_data(df):
    """
    DataFrame의 특정 컬럼들을 변환하고 정규화합니다.
    
    변환:
    - 기초금액 -> log 변환 후 정규화
    - A값 -> 기초금액으로 나눈 후 정규화
    - 투찰률 -> 정규화
    - 참여업체수 -> log 변환 후 정규화
    """
    import numpy as np
    from sklearn.preprocessing import StandardScaler

    # 결과 DataFrame 복사
    result_df = df.copy()
    
    # 스케일러 초기화
    scaler = StandardScaler()
    
    # 1. 기초금액 변환
    result_df['log_기초금액_norm'] = np.log1p(df['기초금액'].astype(float))
    result_df['log_기초금액_norm'] = scaler.fit_transform(result_df[['log_기초금액_norm']])
    
    # 2. A값/기초금액 변환
    result_df['A값/기초금액_norm'] = df['A값'].astype(float) / (1+df['기초금액'].astype(float))
    result_df['A값/기초금액_norm'] = scaler.fit_transform(result_df[['A값/기초금액_norm']])
    
    # 3. 투찰률 정규화
    result_df['투찰률_norm'] = scaler.fit_transform(df[['투찰률(%)']])
    
    # 4. 참여업체수 변환
    result_df['log_참여업체수_norm'] = np.log1p(1+ df['참여업체수'].astype(float))
    result_df['log_참여업체수_norm'] = scaler.fit_transform(result_df[['log_참여업체수_norm']])
    
    # 5. 참여업체수/기초금액 변환
    num_colums = [col for col in columns if col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    for col in num_colums:
        result_df[col] = result_df[col] / result_df['참여업체수']
    
    # 나머지 컬럼들은 이름 유지
    # 나머지 컬럼들은 이름 유지
    keep_columns = [
        '공고번호',  # 식별자이므로 첫 번째로
        'log_기초금액_norm',
        'A값/기초금액_norm',
        '투찰률_norm',
        'log_참여업체수_norm',
        '예가범위',
        '공고구분표시',
        '공고제목',
        '발주처(수요기관)',
        '지역제한',
        '정답사정률(%)'
    ]
    
    # 최종 컬럼 순서 설정
    final_columns = keep_columns + num_colums
    result_df = result_df[final_columns]

    return result_df

처음 13개 컬럼:
Index(['공고번호', '기초금액', 'A값', '투찰률(%)', '참여업체수', '예가범위', '공고구분표시', '공고제목',
       '발주처(수요기관)', '지역제한', '정답사정률(%)', '010_001', '010_002'],
      dtype='object')


In [ ]:
# DataFrame 재구성 실행
raw_data_reorganized = reorganize_dataframe(raw_data)

# normalize 실행
transformed_df = normalize_and_transform_data(raw_data_reorganized)

# CSV 파일로 저장
transformed_df.to_csv(normalized_data_path, index=False)

# Pipeline 함수 정의

In [ ]:
def process_pipeline(df, desired_columns):
    # reorganize data
    organized_df = reorganize_dataframe(df, desired_columns)
    # normalize data
    normalized_df = normalize_and_transform_data(organized_df)
    return organized_df, normalized_df
organized_df, normalized_df = process_pipeline(raw_data, desired_columns)

# save data
organized_df.to_csv(basic_data_path, index=False)
normalized_df.to_csv(normalized_data_path, index=False)